## INFO DASHBOARD

En este documento se encuentran los gráficos que se van a mostrar en el Dashboard final. Estos gráficos mostrarán información extra sobre los alojamientos de Airbnb de NY:
   * PRECIOS
      * Histograma de distribución de precios (apilado barrios)


   * DESCRIPTIVO
     * Pie chart con tipos de casa
     * Spider features sin normalizar
     * Spider features normalizada
     * Bar plot amenities por barrio


   * HOSTS
     * response time y superhost
     * Tabla con información sobre top10 hosts de NY


   * VARIACIÓN DE PRECIOS
     * Gráfico de lineas de Airbnbs con estacionalidad de precios


Todos estos datos serán actualizados con los siguientes filtros que se insertarán en el dashboard final (Dash)
  1. Rentabilidad
  2. Precio por noche 
  3. Barrio

#### Librerías

In [2]:
import numpy as np
from sklearn.decomposition import PCA
from urllib.parse import urlencode
import pandas as pd
import plotly.express as px
import plotly.io as pio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
import math
from plotly.subplots import make_subplots


pio.renderers.default = 'vscode'
pio.templates.default = 'plotly'

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import label_binarize


import plotly.figure_factory as ff

import geopandas 
import json

from cv2 import mean
from itertools import count

import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

#### Importación de datos

In [3]:
listings_total_data = pd.read_parquet("../../Data/total data/modified data/listingsFinalConRentabilidad.parquet")
listings_total_data.head()

,id,last_scraped,host_id,host_name,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,...,price per unit,occupancy_rate,revenue,profitability,amenities_lower,has_wifi,has_dryer,has_tv,has_heating,has_kitchen
0,2539,2022-09-07,2787,John,2008-09-07,within an hour,100,90,f,Gravesend,...,4.418201e+05,0.111236,12139.735955,0.027477,"[""first aid kit"", ""microwave"", ""stove"", ""coffe...",1,1,1,1,1
1,2595,2022-09-07,2845,Jennifer,2008-09-09,within a day,75,23,f,Midtown,...,5.095289e+06,0.398571,25458.750000,0.004997,"[""stove"", ""coffee maker"", ""long term stays all...",1,1,1,1,1
2,5121,2022-09-07,7356,Garon,2009-02-03,within an hour,100,100,t,Bedford-Stuyvesant,...,2.552823e+05,0.333134,7295.641791,0.028579,"[""heating"", ""kitchen"", ""air conditioning"", ""wi...",1,0,0,1,1
3,45910,2022-09-07,204539,Mark,2010-08-17,within an hour,100,19,f,Ridgewood,...,3.830991e+05,0.098630,15300.000000,0.039937,"[""hair dryer"", ""essentials"", ""carbon monoxide ...",1,1,1,1,1
4,5136,2022-09-07,7378,Rebecca,2009-02-03,None,None,33,f,Greenwood Heights,...,3.286703e+05,0.042235,4239.301676,0.012898,"[""hair dryer"", ""cable tv"", ""refrigerator"", ""bb...",1,1,1,1,1


In [27]:
# lectura de base de datos de hosts
hosts_df = pd.read_parquet("../../Data/total data/modified data/hosts_df.parquet")
hosts_df.head()

,ID host,Nombre host,Total de Airbnbs,Nota media de valoración,Años de antigüedad,Superhost,Total de distritos con casa,Total de barrios con casa,Media de huéspedes,Media de camas,Media de baños,Precio medio por noche,Porcentaje de casas sobre el total de NY
5575,158969505,Untitled,184,4.7,5.0,Sí,2,1,2.1,1.0,1.0,301.8,1.20
3864,51501835,Jeniffer,108,4.4,6.9,No,4,1,3.0,1.2,1.0,133.5,0.71
7581,371972456,Justin,99,4.7,2.1,Sí,1,1,2.5,1.3,1.0,407.0,0.65
5275,137358866,Kaz,98,4.6,5.4,Sí,11,3,1.4,1.0,1.1,79.5,0.64
4092,61391963,Stay With Vibe,91,4.5,6.7,No,7,1,2.2,1.2,1.0,155.9,0.60


In [30]:
# lectura de datos de fechas
ListingDiffSemanaFinde = pd.read_parquet("../../Data/total data/modified data/calendario.parquet")
ListingDiffSemanaFindePlotear = pd.read_parquet("../../Data/total data/modified data/paraPlotearCalendario.parquet")

In [4]:
# lectura de geoson distritos
geosonNeighbourhoods = geopandas.read_file("../../Data/neighbourhoods/neighbourhoods.geojson")
jsonGeoNeigh = json.load(open("../../Data/neighbourhoods/neighbourhoods.geojson"))
geosonNeighbourhoods

,neighbourhood,neighbourhood_group,geometry
0,Bayswater,Queens,"MULTIPOLYGON (((-73.76671 40.61491, -73.76825 ..."
1,Allerton,Bronx,"MULTIPOLYGON (((-73.84860 40.87167, -73.84582 ..."
2,City Island,Bronx,"MULTIPOLYGON (((-73.78282 40.84392, -73.78257 ..."
3,Ditmars Steinway,Queens,"MULTIPOLYGON (((-73.90160 40.76777, -73.90391 ..."
4,Ozone Park,Queens,"MULTIPOLYGON (((-73.83754 40.69136, -73.83108 ..."
...,...,...,...
228,Westchester Square,Bronx,"MULTIPOLYGON (((-73.84294 40.84640, -73.83913 ..."
229,Westerleigh,Staten Island,"MULTIPOLYGON (((-74.13131 40.62633, -74.13192 ..."
230,Williamsbridge,Bronx,"MULTIPOLYGON (((-73.86722 40.86969, -73.86701 ..."
231,Williamsburg,Brooklyn,"MULTIPOLYGON (((-73.95757 40.72510, -73.95300 ..."


In [5]:
# lectura de geoson barrios
bigJSONNeigh = json.load(open("../../Data/neighbourhoods/bigneighbourhoods.geojson"))
geosonNeighbourhoodsBig = geopandas.read_file("../../Data/neighbourhoods/bigneighbourhoods.geojson")
geosonNeighbourhoodsBig

,neighbourhood_group,geometry
0,Brooklyn,"POLYGON ((-73.98202 40.57139, -73.98199 40.571..."
1,Bronx,"MULTIPOLYGON (((-73.89015 40.80496, -73.88937 ..."
2,Staten Island,"POLYGON ((-74.21338 40.50677, -74.21353 40.506..."
3,Queens,"MULTIPOLYGON (((-73.86269 40.56650, -73.86270 ..."
4,Manhattan,"MULTIPOLYGON (((-73.99955 40.70769, -73.99961 ..."


#### PRECIOS


Histograma de distribución de precios (apilado barrios)

In [10]:
data = []
colorsBarrios = {
    "Brooklyn": "#1f77b4",
    "Bronx": "#ff7f0e",
    "Staten Island":"#2ca02c",
    "Queens":"#9467bd",
    "Manhattan":"#d62728"
}

for x in listings_total_data["neighbourhood_group_cleansed"].unique():
    data.append(go.Histogram(
                    x = listings_total_data[listings_total_data["neighbourhood_group_cleansed"] == x]['price'],
                    marker_color=colorsBarrios[x],
                    xbins=dict(
                        start= 0,
                        end= 600,
                        size=5
                    ),
                    opacity=0.5,
                    name = x
                )
    )
    
layout = go.Layout(title = "<b>Distribución de los precios por barrios<b>", xaxis_title = "Precios por noche", yaxis_title = "Frecuencia",
                barmode = "overlay", bargap = 0.1, height=450, width=1500, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', font = dict(color = 'white', size=12))

fig = go.Figure(data = data, layout = layout)
fig.show()

#### DESCRIPTIVO

* Pie chart con tipos de casa

*Se han elegido los tipos de casa más observados en el dataset*

In [13]:
tiposDeListing = ["Entire rental unit", "Private room in rental unit", "Private room in home", "Entire condo", "Entire home"]
fig = px.pie(data_frame=listings_total_data[listings_total_data['property_type'].isin(tiposDeListing)].groupby("property_type",as_index=False).count(), 
             values="id",names="property_type", color = "property_type",color_discrete_sequence = px.colors.sequential.YlGnBu,
             )
fig.update_layout(title=dict(text="<b>Tipos de viviendas en NYC<b>", x=0.20,y=0.97, font=dict(size=17)),paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', font = dict(color = 'white', size=15), height=400,width=734)
fig.show()

* Spider features sin normalizar

In [15]:
colorsBarrios = {
        "Brooklyn": "#1f77b4",
        "Bronx": "#ff7f0e",
        "Staten Island":"#2ca02c",
        "Queens":"#9467bd",
        "Manhattan":"#d62728"
    }

lista = ["occupancy_rate",
    "beds",
    "price",
    "baths",
    "review_scores_rating",
    "profitability"]

maximo = {}
for y in lista:
    maximo[y]=(max(listings_total_data.groupby("neighbourhood_group_cleansed").agg('mean')[y]))

fig = go.Figure()
lista.append(lista[0])


for x in listings_total_data["neighbourhood_group_cleansed"].unique():
    data = []
    for y in lista:
        data.append((listings_total_data[listings_total_data["neighbourhood_group_cleansed"]==x].groupby("neighbourhood_group_cleansed").agg('mean')[y][0])/maximo[y])
        #print((df[df["neighbourhood_group_cleansed"]==x].groupby("neighbourhood_group_cleansed").agg('mean')[y][0]))
    #data.append(data[0])

    
    fig.add_trace(go.Scatterpolar(
                r=data,
                theta=lista,
                mode='lines',
                line_color=colorsBarrios[x],
                name=x,
                )
    )


fig.update_layout(
polar=dict(
    radialaxis=dict(
    visible=True
    ),
    
),
showlegend=True,
)

fig.update_layout(title=dict(text="<b>Valores medios por cada geografía</b><br><i>No se filtra por Barrio para poder verlos todos</i>", x=0.50,y=0.97, font=dict(size=17)),paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', font = dict(color = 'white', size=15), height=400,width=734)

fig.show()

* Spider features normalizada


In [17]:
colorsBarrios = {
        "Brooklyn": "#1f77b4",
        "Bronx": "#ff7f0e",
        "Staten Island":"#2ca02c",
        "Queens":"#9467bd",
        "Manhattan":"#d62728"
    }

lista = ["occupancy_rate",
    "beds",
    "price",
    "baths",
    "review_scores_rating",
    "profitability"]


fig = go.Figure()

maximo = {}
for y in lista:
    maximo[y]=(np.mean(listings_total_data.groupby("neighbourhood_group_cleansed").agg('mean')[y]),
                (listings_total_data.groupby("neighbourhood_group_cleansed").agg('mean')[y].std()),
                max(listings_total_data.groupby("neighbourhood_group_cleansed").agg('mean')[y]))

lista.append(lista[0])

for x in listings_total_data["neighbourhood_group_cleansed"].unique():
    data = []
    for y in lista:
        #data.append((listings_filtered_df[listings_filtered_df["neighbourhood_group_cleansed"]==x].groupby("neighbourhood_group_cleansed").agg('mean')[y][0])/maximo[y])
        data.append((listings_total_data[listings_total_data["neighbourhood_group_cleansed"]==x].groupby("neighbourhood_group_cleansed").agg('mean')[y][0]-maximo[y][0])/maximo[y][1])
    data.append(data[0])
    
    fig.add_trace(go.Scatterpolar(
                r=data,
                theta=lista,
                mode='lines',
                line_color=colorsBarrios[x],
                name=x,
                
                )
    )

fig.update_layout(
polar=dict(
    radialaxis=dict(
    visible=True
    ),
    
),
showlegend=True,
)

fig.update_layout(title=dict(text="<b>Valores medios normalizados por cada geografía</b><br><i>No se aplican filtros al gráfico<i>", x=0.50,y=0.97, font=dict(size=17)),paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', font = dict(color = 'white', size=15), height=400,width=734)
fig.show()

* Bar plot amenities por barrio

In [19]:
total_neighbourhood = listings_total_data.groupby("neighbourhood_group_cleansed").agg({"id":"count"}).reset_index()
total_neighbourhood = total_neighbourhood.rename({"id":"total"},axis=1)

grouped_amenities_df = listings_total_data.groupby("neighbourhood_group_cleansed").agg({"has_wifi":"sum", "has_dryer":"sum", "has_tv":"sum", "has_heating":"sum", "has_kitchen":"sum"})
grouped_amenities_df = grouped_amenities_df.reset_index()

join_df = pd.merge(grouped_amenities_df,total_neighbourhood, how="left",on="neighbourhood_group_cleansed")

join_df["percentage_has_wifi"] = (join_df["has_wifi"]/join_df["total"]).round(2)
join_df["percentage_has_dryer"] = (join_df["has_dryer"]/join_df["total"]).round(2)
join_df["percentage_has_tv"] = (join_df["has_tv"]/join_df["total"]).round(2)
join_df["percentage_has_heating"] = (join_df["has_heating"]/join_df["total"]).round(2)
join_df["percentage_has_kitchen"] = (join_df["has_kitchen"]/join_df["total"]).round(2)

newnames = {'percentage_has_wifi':'Tiene wifi', 'percentage_has_dryer': 'Tiene secador', 'percentage_has_tv': 'Tiene TV', 'percentage_has_heating': 'Tiene calefacción', 'percentage_has_kitchen': 'Tiene cocina'}
join_df = join_df.rename(newnames,axis=1)
join_df

fig = go.Figure()
fig = px.bar(join_df, x="neighbourhood_group_cleansed", y=["Tiene wifi","Tiene secador", "Tiene TV", "Tiene calefacción", "Tiene cocina"], barmode='group', color_discrete_sequence=px.colors.diverging.Temps)
fig.update_layout(title = dict(text="<b>Porcentaje de Aribnbs por barrios que presentan los servicios más demandados</b><br><i>No se aplican filtros a este gráfico<i>", font=dict(size=17), x=0.5,y=0.95),
                    xaxis_title ="",yaxis_title = "Porcentaje de Airbnbs (%)", paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', font = dict(color = 'white', size=15), height=500,width=1500,

                    legend= dict(title=""))
fig.show()

#### HOSTS

* Sankey response time y superhost

In [25]:
diccionarioColoresTiempo={
    #dusty teal
    "a few days or more":"#1BA597",
    "within a day":"#AEE78D",
    "within a few hours":"#F4F39B",
    "within an hour":"#F3CE88"
}


level_count = pd.DataFrame(listings_total_data.groupby("host_response_time")["host_is_superhost"].value_counts()).rename(columns = {"host_is_superhost": "count"}).reset_index()
group_count = pd.DataFrame(level_count.groupby(["host_response_time"])["count"].sum()).reset_index()
level_count=level_count.merge(group_count, on='host_response_time', how='left').rename(columns = {"count_x": "count","count_y": "total"})
level_count

nivelesIntermedios = level_count["host_response_time"].unique().tolist()
lista1 = []
lista2 = []
lista3 = []
lista7 = []


for index,tiempo in enumerate(nivelesIntermedios):
    lista1.append(index+1)
    lista2.append(len(nivelesIntermedios)+1)
    #print(level_count[(level_count["host_response_time"]==tiempo)&(level_count["host_is_superhost"]=='t')]["count"].values)
    print()
    
    try:
        lista3.append(level_count[(level_count["host_response_time"]==tiempo)&(level_count["host_is_superhost"]=='t')]["count"].values[0])
    except:
        lista3.append(0)
    
    lista7.append("gold")
    
    lista1.append(index+1)
    lista2.append(len(nivelesIntermedios)+2)
    #print(level_count[(level_count["host_response_time"]==tiempo)&(level_count["host_is_superhost"]=='f')]["count"].values)
    try:
        lista3.append(level_count[(level_count["host_response_time"]==tiempo)&(level_count["host_is_superhost"]=='f')]["count"].values[0])
    except:
        lista3.append(0)
    
    
    lista7.append("lightgray")
lista4 = []
lista5 = []
lista6 = []

for index,tiempo in enumerate(nivelesIntermedios):
    lista4.append(0)
    lista5.append(index+1)
    lista6.append(level_count[(level_count["host_response_time"]==tiempo)]["total"].values[0])
    lista7.append(diccionarioColoresTiempo[tiempo])

fig = go.Figure(data=[go.Sankey(
    node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = ["Todos los hosts"]+nivelesIntermedios+["SuperHost","Host"],
        color = "white"
    ),
    link = dict(
        source = lista1+lista4, 
        target = lista2+lista5,
        value = lista3+lista6,
        color= lista7
    ))])

# fig.update_layout(title="Distribucion tiempo de respuesta para host y superhost", 
#                   font_size=20,font_color="white",paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)',
#                   height=800,width=2200)

fig.update_layout(height=500,width=1500,paper_bgcolor='rgba(0,0,0,0)', title = dict(text="<b>Distribución tiempo de respuesta para host y superhost<b>", x=0.5,y=0.95,font=dict(size=17,color='white')), plot_bgcolor='rgba(0,0,0,0)', font = dict(color = 'white', size=17))    

fig.show()

* Tabla con información sobre top10 hosts de NY

In [29]:
filtered_data = hosts_df.nlargest(10, 'Total de Airbnbs')
filtered_data = filtered_data.drop(["ID host","Nombre host"],axis=1)
column_host = ["Host 1", "Host 2", "Host 3", "Host 4", "Host 5", "Host 6", "Host 7", "Host 8", "Host 9", "Host 10"]
filtered_data.insert(0, "Host",column_host)

fig = go.Figure()

fig.add_trace(
    go.Table(
        header=dict(
            values=filtered_data.columns,
            line_color='white',
            fill_color='#4D5656',
            font=dict(size=17, color="#D7DBDD"),
            align=['center'],
        ),
        cells=dict(
            values= [filtered_data[k].tolist() for k in filtered_data.columns],
            align = ['center'],
            #fill=dict(color=['paleturquoise', 'white']),
            font=dict(size=16, color="#4D5656"),
            height=30
        )

    ),
)

fig.update_layout(height=500,width=1500,paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', font = dict(color = 'white', size=18), 
                title = dict(text="<b>Información sobre los top 10 host de Nueva York<b>",x=0.5,y=0.90, font=dict(size=17)))

fig.show()

#### VARIACIÓN DE PRECIOS

* Gráfico de lineas de Airbnbs con estacionalidad de precios

In [33]:
listaCambiantes = [16595,16821,13808,14290,23135,29683,2595,39282,23686,53469,57297,53470,59121]

fig = px.line(data_frame=ListingDiffSemanaFinde[(ListingDiffSemanaFinde["listing_id"].isin(listaCambiantes))], x='date', y='priceNum', color='listing_id')
fig.update_layout(height=500,width=1500,paper_bgcolor='rgba(0,0,0,0)', legend= dict(title="ID Airbnb"), xaxis_title = "", yaxis_title="Precio por noche ($)", title = dict(text="<b>Demostración de patrones de evolución del precio de algunos Airbnbs<b>", x=0.5,y=0.95,font=dict(size=17,color='white')), plot_bgcolor='rgba(0,0,0,0)', font = dict(color = 'white', size=15))    

fig.show()